In [ ]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import pygame
import threading
import time
from collections import deque
import tkinter as tk
from tkinter import ttk
import pyttsx3
from tkinter import messagebox
import queue
import folium
import webbrowser
import os
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import requests
import json
from math import radians, sin, cos, sqrt, atan2

class DrowsinessDetectionSystem:
    def __init__(self):
        # Initialize audio
        pygame.mixer.init()
        self.alarm_sound = pygame.mixer.Sound(r'D:\7th_sem\Driver_Drowsiness\first_alarm.mp3')
        self.model = load_model(r"D:\7th_sem\Driver_Drowsiness\model2.h5")
        
        # Initialize pyttsx3 with error handling
        try:
            self.engine = pyttsx3.init('sapi5')
            self.engine.setProperty('rate', 150)
            # Test the engine
            voices = self.engine.getProperty('voices')
            if voices:
                self.engine.setProperty('voice', voices[0].id)
            print("Speech engine initialized successfully")
        except Exception as e:
            print(f"Error initializing speech engine: {e}")
            self.engine = None
            
        # Initialize cascades
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
        
        # Initialize variables
        self.driver_name = None
        self.system_choice = None
        self.from_city = None
        self.to_city = None
        self.alarm_playing = False
        self.eye_closed_start_time = None
        self.warning_given = False
        self.prediction_window = deque(maxlen=10)
        self.speech_thread_running = True
        self.is_speaking = False
        self.map_shown = False
        
        # Initialize geocoder
        self.geolocator = Nominatim(user_agent="drowsiness_detection_app")
        
        # Add debug flag
        self.debug = True
        
        # Initialize speech queue
        self.speech_queue = queue.Queue()

    def ai_speech_thread(self):
        """
        Thread to handle text-to-speech queue for AI system warnings
        """
        print("Speech thread started")  # Debug print
        while self.speech_thread_running:
            try:
                if not self.is_speaking and not self.speech_queue.empty():
                    text = self.speech_queue.get()
                    print(f"Attempting to speak: {text}")  # Debug print
                    self.is_speaking = True
                    
                    if self.engine is not None:
                        try:
                            # Force wait for previous speech to complete
                            if self.engine._inLoop:
                                self.engine.endLoop()
                            
                            self.engine.say(text)
                            self.engine.runAndWait()
                        except Exception as e:
                            print(f"Speech engine error: {e}")
                    else:
                        print("Speech engine not initialized")
                    
                    self.is_speaking = False
                    print("Speech completed")  # Debug print
                
                time.sleep(0.1)
                
            except Exception as e:
                print(f"Error in speech thread: {e}")
                self.is_speaking = False
                time.sleep(0.1)

    def play_buzzer(self):
        """
        Play alarm sound for buzzer system
        """
        if not self.alarm_playing:
            self.alarm_playing = True
            self.alarm_sound.play(-1)

    def stop_buzzer(self):
        """
        Stop alarm sound
        """
        if self.alarm_playing:
            self.alarm_playing = False
            self.alarm_sound.stop()

    def get_current_location(self):
        """
        Calculate approximate current location between from_city and to_city
        """
        try:
            from_location = self.geolocator.geocode(self.from_city)
            to_location = self.geolocator.geocode(self.to_city)
            
            if from_location and to_location:
                lat1, lon1 = from_location.latitude, from_location.longitude
                lat2, lon2 = to_location.latitude, to_location.longitude
                
                mid_lat = (lat1 + lat2) / 2
                mid_lon = (lon1 + lon2) / 2
                
                return (mid_lat, mid_lon)
            return None
        except Exception as e:
            print(f"Error getting location: {e}")
            return None

    def find_nearby_coffee_shops(self, latitude, longitude, radius=50000):  # 50km radius
        """
        Find nearby coffee shops using OpenStreetMap's Overpass API
        """
        overpass_url = "http://overpass-api.de/api/interpreter"
        query = f"""
        [out:json];
        (
          node["amenity"="cafe"](around:{radius},{latitude},{longitude});
          node["shop"="coffee"](around:{radius},{latitude},{longitude});
          way["amenity"="cafe"](around:{radius},{latitude},{longitude});
          way["shop"="coffee"](around:{radius},{latitude},{longitude});
          relation["amenity"="cafe"](around:{radius},{latitude},{longitude});
          relation["shop"="coffee"](around:{radius},{latitude},{longitude});
        );
        out center;
        """
        
        try:
            response = requests.post(overpass_url, data=query)
            data = response.json()
            
            coffee_shops = []
            for element in data.get('elements', []):
                if element.get('type') == 'node':
                    lat = element.get('lat')
                    lon = element.get('lon')
                else:
                    center = element.get('center', {})
                    lat = center.get('lat')
                    lon = center.get('lon')
                
                if lat and lon:
                    name = element.get('tags', {}).get('name', 'Coffee Shop')
                    coffee_shops.append({
                        'name': name,
                        'latitude': lat,
                        'longitude': lon,
                        'distance': geodesic((latitude, longitude), (lat, lon)).kilometers
                    })
            
            return coffee_shops
        except Exception as e:
            print(f"Error finding coffee shops: {e}")
            return []

    def create_map(self, current_location, coffee_shops):
        """
        Create and save an HTML map with working route display functionality
        """
        try:
            if not current_location:
                print("Error: No current location provided")
                return None
                
            # Create map centered on current location
            m = folium.Map(location=current_location, zoom_start=11)
            
            # Add current location marker
            folium.Marker(
                current_location,
                popup="Your Current Location",
                icon=folium.Icon(color='red', icon='info-sign'),
                tooltip="You are here"
            ).add_to(m)
            
            # Add coffee shop markers
            for shop in coffee_shops:
                popup_content = f"""
                <div style="font-family: Arial; min-width: 150px;">
                    <h4 style="margin: 0; color: #2c3e50;"><b>☕ {shop['name']}</b></h4>
                    <p style="margin: 5px 0;">Distance: {shop['distance']:.1f} km</p>
                    <button onclick="showRoute({shop['latitude']}, {shop['longitude']});"
                            style="background-color: #3498db; color: white; border: none; 
                                   padding: 5px 10px; border-radius: 3px; cursor: pointer;">
                        Show Route
                    </button>
                </div>
                """
                
                folium.Marker(
                    [shop['latitude'], shop['longitude']],
                    popup=folium.Popup(popup_content, max_width=200),
                    icon=folium.Icon(color='green', icon='info-sign', prefix='fa'),
                    tooltip="Coffee Shop"
                ).add_to(m)
            
            # Add fixed route functionality
            m.get_root().html.add_child(folium.Element("""
                <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet-polylinedecorator/1.6.0/leaflet.polylineDecorator.min.js"></script>
                <script>
                var currentMap = document.querySelector('#map');  // Get the map element
                var route = null;
                var arrowDecorator = null;
                
                function showRoute(lat, lng) {
                    // Get the map instance
                    var map = currentMap._leaflet_map || window.map;
                    
                    // Remove existing route and decorator if they exist
                    if (route) {
                        map.removeLayer(route);
                    }
                    if (arrowDecorator) {
                        map.removeLayer(arrowDecorator);
                    }
                    
                    // Create route coordinates
                    var startPoint = [%f, %f];  // Current location
                    var endPoint = [lat, lng];  // Destination
                    
                    // Create route outline (white border)
                    var routeOutline = L.polyline([startPoint, endPoint], {
                        color: 'white',
                        weight: 10,
                        opacity: 0.7
                    }).addTo(map);
                    
                    // Create main route line
                    route = L.polyline([startPoint, endPoint], {
                        color: '#FF4500',
                        weight: 6,
                        opacity: 1
                    }).addTo(map);
                    
                    // Ensure proper layering
                    routeOutline.bringToBack();
                    route.bringToFront();
                    
                    // Add arrow decorator
                    arrowDecorator = L.polylineDecorator(route, {
                        patterns: [
                            {
                                offset: '25%%',
                                repeat: 50,
                                symbol: L.Symbol.arrowHead({
                                    pixelSize: 15,
                                    polygon: true,
                                    pathOptions: {
                                        color: '#FF4500',
                                        fillColor: '#FF4500',
                                        fillOpacity: 1,
                                        weight: 2
                                    }
                                })
                            }
                        ]
                    }).addTo(map);
                    
                    // Fit map bounds to show entire route
                    var bounds = L.latLngBounds([startPoint, endPoint]);
                    map.fitBounds(bounds, {
                        padding: [50, 50],
                        maxZoom: 15
                    });
                }
                </script>
            """ % (current_location[0], current_location[1])))
            
            # Add legend
            legend_html = """
            <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; 
                        background-color: white; padding: 10px; border: 2px solid grey; 
                        border-radius: 5px; font-family: Arial;">
                <div style="margin-bottom: 5px;">
                    <i class="fa fa-info-sign fa-2x" style="color: red;"></i> 
                    <span style="vertical-align: super; margin-left: 5px;">Your Location</span>
                </div>
                <div style="margin-bottom: 5px;">
                    <i class="fa fa-info-sign fa-2x" style="color: green;"></i> 
                    <span style="vertical-align: super; margin-left: 5px;">Coffee Shops</span>
                </div>
                <div>
                    <span style="color: #FF4500; font-size: 20px;">➜</span> 
                    <span style="vertical-align: super; margin-left: 5px;">Route to Shop</span>
                </div>
            </div>
            """
            m.get_root().html.add_child(folium.Element(legend_html))
            
            # Save map
            map_path = os.path.join(os.path.expanduser("~"), "drowsiness_map.html")
            m.save(map_path)
            
            return map_path
            
        except Exception as e:
            print(f"Error creating map: {str(e)}")
            return None
        
    def show_map(self):
        """
        Show the map in a web browser with error handling and debugging
        """
        try:
            print("Attempting to show map...")
            
            if not self.map_shown:
                # Get current location with error checking
                current_location = self.get_current_location()
                if not current_location:
                    print("Error: Could not determine current location")
                    return
                    
                print(f"Current location obtained: {current_location}")
                
                # Find coffee shops with error checking
                coffee_shops = self.find_nearby_coffee_shops(*current_location)
                if not coffee_shops:
                    print("Warning: No coffee shops found in the area")
                    return
                    
                print(f"Found {len(coffee_shops)} coffee shops")
                
                # Create map with error checking
                map_path = self.create_map(current_location, coffee_shops)
                if not map_path:
                    print("Error: Could not create map")
                    return
                    
                print(f"Map created at: {map_path}")
                
                # Verify map file exists
                if not os.path.exists(map_path):
                    print(f"Error: Map file not found at {map_path}")
                    return
                    
                # Open map in browser with absolute path
                map_url = 'file:///' + os.path.abspath(map_path).replace('\\', '/')
                print(f"Opening map URL: {map_url}")
                
                webbrowser.open(map_url)
                self.map_shown = True
                
                if coffee_shops:
                    # Ensure we wait a moment for the map to open
                    time.sleep(1)
                    
                    # Single navigation prompt with clear enunciation
                    navigation_prompt = ("Please choose any one of the nearby coffee shops and click the Show Route "
                                      "button to see the navigation path to your selected destination.")
                    
                    # Force a new speech attempt
                    if self.engine is not None:
                        try:
                            print("Attempting to speak navigation prompt...")
                            self.is_speaking = True
                            self.engine.say(navigation_prompt)
                            self.engine.runAndWait()
                            self.is_speaking = False
                            print("Navigation prompt spoken successfully")
                        except Exception as e:
                            print(f"Error speaking navigation prompt: {e}")
                            self.is_speaking = False
                    else:
                        print("Speech engine not available")
                    
        except Exception as e:
            print(f"Error showing map: {str(e)}")
    
    def handle_drowsiness(self, smoothed_eyes_open):
        """
        Handle drowsiness detection and trigger appropriate alerts
        """
        current_time = time.time()
    
        if not smoothed_eyes_open:
            if self.eye_closed_start_time is None:
                self.eye_closed_start_time = current_time
                self.warning_given = False
            
            elapsed_time = current_time - self.eye_closed_start_time
    
            if elapsed_time >= 2.0 and not self.warning_given:
                if self.system_choice == 'ai':
                    if self.debug:
                        print(f"Triggering AI warning at {elapsed_time:.1f} seconds")
                    warning_message = (
                        f"Hey {self.driver_name}! We have detected drowsiness in your eyes. "
                        "Please check a nearby coffee shop on the map or find a rest place "
                        "so that you can be actively ready to continue your journey."
                    )
                    self.speech_queue.put(warning_message)
                    self.warning_given = True
                    
                    # Show map in a separate thread to avoid blocking
                    map_thread = threading.Thread(target=self.show_map, daemon=True)
                    map_thread.start()
                    
                elif self.system_choice == 'buzzer':
                    self.play_buzzer()
    
            if self.debug:
                print(f"Eyes closed for {elapsed_time:.1f} seconds. Warning given: {self.warning_given}")
        else:
            if self.eye_closed_start_time is not None and self.debug:
                print("Eyes opened - resetting warnings")
            self.eye_closed_start_time = None
            self.warning_given = False
            self.map_shown = False
            if self.system_choice == 'buzzer':
                self.stop_buzzer()

    def initialize_camera(self):
        """
        Initialize and configure the camera
        """
        camera = cv2.VideoCapture(0)
        camera.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        camera.set(cv2.CAP_PROP_FPS, 30)
        time.sleep(2)
        return camera

    def detect_eyes(self, frame, gray):
        """
        Detect and analyze eyes in the frame
        """
        eyes_open = False
        faces = self.face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
        
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            eyes = self.eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=3)

            for (ex, ey, ew, eh) in eyes:
                eye = frame[y + ey:y + ey + eh, x + ex:x + ex + ew]
                if eye.size > 0:
                    eye = cv2.resize(eye, (96, 96)) / 255.0
                    eye = np.expand_dims(eye, axis=0)
                    prediction = self.model.predict(eye, verbose=0)

                    if prediction[0][0] > 0.5:
                        eyes_open = True
                        cv2.rectangle(frame, (x + ex, y + ey), (x + ex + ew, y + ey + eh), (0, 255, 0), 2)
                    else:
                        cv2.rectangle(frame, (x + ex, y + ey), (x + ex + ew, y + ey + eh), (0, 0, 255), 2)
        
        return eyes_open, frame

    def run_detection(self):
        """
        Main detection loop
        """
        camera = self.initialize_camera()
        
        if not camera.isOpened():
            print("Error: Could not open camera.")
            return
        
        try:
            while True:
                ret, frame = camera.read()
                if not ret:
                    print("Failed to capture frame. Reinitializing camera...")
                    camera.release()
                    camera = self.initialize_camera()
                    continue

                frame = cv2.resize(frame, (320, 240))
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                eyes_open, frame = self.detect_eyes(frame, gray)

                self.prediction_window.append(1 if eyes_open else 0)
                open_percentage = sum(self.prediction_window) / len(self.prediction_window)
                smoothed_eyes_open = open_percentage >= 0.5

                status = "OPEN" if smoothed_eyes_open else "CLOSED"
                color = (0, 255, 0) if smoothed_eyes_open else (0, 0, 255)
                
                if not smoothed_eyes_open and self.eye_closed_start_time is not None:
                    elapsed_time = time.time() - self.eye_closed_start_time
                    cv2.putText(frame, f"Eyes: {status} ({elapsed_time:.1f}s)", 
                              (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                else:
                    cv2.putText(frame, f"Eyes: {status}", 
                              (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

                self.handle_drowsiness(smoothed_eyes_open)

                cv2.imshow('Driver Drowsiness Detection', cv2.resize(frame, (640, 480)))

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        finally:
            camera.release()
            cv2.destroyAllWindows()
            self.cleanup()

    def cleanup(self):
        """
        Clean up resources before exiting
        """
        self.speech_thread_running = False
        time.sleep(1)
        pygame.mixer.quit()

    def get_driver_details(self):
        """
        Create GUI for getting driver details
        """
        def on_ok():
            self.driver_name = name_entry.get()
            self.from_city = from_entry.get()
            self.to_city = to_entry.get()
            self.system_choice = choice_var.get()

            if not all([self.driver_name, self.from_city, self.to_city, self.system_choice]):
                messagebox.showerror("Error", "Please fill in all fields and select a detection system.")
                return

            # Start speech thread before destroying root
            speech_thread = threading.Thread(target=self.ai_speech_thread, daemon=True)
            speech_thread.start()

            # Wait a moment for thread to initialize
            time.sleep(2)

            # Queue welcome message
            welcome_message = (f"Hi, {self.driver_name}! Welcome to the system for detecting driver drowsiness. "
                             f"I wish you a good journey from {self.from_city} to {self.to_city}. Please drive carefully.")
            print("Queueing welcome message")  # Debug print
            self.speech_queue.put(welcome_message)

            # Destroy root window after queuing message
            root.destroy()

            # Start detection thread
            detection_thread = threading.Thread(target=self.run_detection)
            detection_thread.start()

        root = tk.Tk()
        root.title("Driver Details")
        root.geometry("400x350")
        root.resizable(False, False)

        # Center window
        root.update_idletasks()
        width = root.winfo_width()
        height = root.winfo_height()
        x = (root.winfo_screenwidth() // 2) - (width // 2)
        y = (root.winfo_screenheight() // 2) - (height // 2)
        root.geometry('{}x{}+{}+{}'.format(width, height, x, y))

        main_frame = ttk.Frame(root, padding="20")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        main_frame.columnconfigure(0, weight=1)
        main_frame.columnconfigure(1, weight=1)

        ttk.Label(main_frame, text="Driver Name:").grid(column=0, row=0, sticky=tk.W, pady=5)
        name_entry = ttk.Entry(main_frame, width=30)
        name_entry.grid(column=1, row=0, pady=5, sticky=(tk.W, tk.E))

        ttk.Label(main_frame, text="From:").grid(column=0, row=1, sticky=tk.W, pady=5)
        from_entry = ttk.Entry(main_frame, width=30)
        from_entry.grid(column=1, row=1, pady=5, sticky=(tk.W, tk.E))

        ttk.Label(main_frame, text="To:").grid(column=0, row=2, sticky=tk.W, pady=5)
        to_entry = ttk.Entry(main_frame, width=30)
        to_entry.grid(column=1, row=2, pady=5, sticky=(tk.W, tk.E))

        ttk.Label(main_frame, text="Choose any one Drowsiness Detection System:").grid(
            column=0, row=3, columnspan=2, sticky=tk.W, pady=(20, 5))

        choice_var = tk.StringVar(value='ai')
        ttk.Radiobutton(main_frame, text="AI Recognition Speech", variable=choice_var, 
                       value='ai').grid(column=0, row=4, columnspan=2, sticky=tk.W, pady=5)
        ttk.Radiobutton(main_frame, text="Buzzer System", variable=choice_var,
                       value='buzzer').grid(column=0, row=5, columnspan=2, sticky=tk.W, pady=5)

        ttk.Button(main_frame, text="OK", command=on_ok).grid(column=0, row=6, columnspan=2, pady=20)

        root.mainloop()

if __name__ == "__main__":
    system = DrowsinessDetectionSystem()
    system.get_driver_details()

pygame 2.6.0 (SDL 2.28.4, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
Speech engine initialized successfully
